## Power transmission line impedance calculator

For three-phase, balanced transmission systems.  Reports positive, negative, and zero sequence symmetrical components of line impedance.  Lines are positioned as shown below with x and y coordinates and resistivity is selected based on type of earth below the line.  Method is based on Power System Analysis and Design [1] pages 197 on.  Earth is replaced with earth return conductors as shown in the Glover textbook referencing Carson [2]

<img src="transmission line positions.png" alt="diagram" style="width:300px;"> <img src="earth resistivities.png" alt="earth resistivities" style="width:500px">

[1] J. D. Glover, T. J. Overbye, and M. S. Sarma, Power System Analysis & Design. Boston, MA: Cengage Learning, 2017. 

[2] John R. Carson, “Wave Propagation in Overhead Wires with Ground
Return,” Bell System Tech. J. 5 (1926): 539–554.

In [149]:
import numpy as np
import cmath

m_to_ft = lambda m : m * 3.28084
ft_to_m = lambda ft : ft * 0.30479999
mi_to_m = lambda mi : mi * 1.60934 * 1000
km_to_mi = lambda km : km * 0.621371

# System parameters
f = 60 # Hz

# Condcutor positions (feet)
apos = 0, 48
bpos = 25, 48
cpos = 50, 48
n1pos = 10, 48+23
n2pos = 40, 48+23

# Conductor properties
Ra = Rb = Rc = 0.1128           # Ohms per mile per conductor
GMRa = GMRb = GMRc = 0.0403     # Geometric mean radius (feet)
bundle = 2                      # Conductors in bundle
bundle_spacing = 1.5            # feet

# Neutral conductor properties
Rn1 = Rn2 = 1.52                # Ohms per km
GMRn1 = GMRn2 = 0.0021          # feet

# values for dry earth
Dkkp = m_to_ft(2690)    # feet

In [150]:
def dist(x1, y1, x2, y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

Rkp = 9.869E-7*f
Rk = (Ra / mi_to_m(1)) / bundle     # convert to ohms per meter per phase
Rn = (Rn1 / 1000)                   # convert to ohms per meter

Dkkc = (GMRa * bundle_spacing**(bundle-1))**(1/(bundle))
Dkkg = GMRn1 

Zaa = Zbb = Zcc = Rk + Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log(Dkkp/Dkkc))
Zn1n1 = Zn2n2 = Rn + Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log(Dkkp/GMRn1))

Zab = Zbc =            Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(apos[0], apos[1], bpos[0], bpos[1]-Dkkp))/(dist(apos[0],apos[1],bpos[0],bpos[1]))))
Zac =                  Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(apos[0],apos[1], cpos[0], cpos[1]-Dkkp))/(dist(apos[0],apos[1],cpos[0],cpos[1]))))

Zan1 = Zcn2 =          Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(apos[0], apos[1], n1pos[0], n1pos[1]-Dkkp))/(dist(apos[0],apos[1],n1pos[0],n1pos[1]))))
Zan2 = Zcn1 =          Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(apos[0], apos[1], n2pos[0], n2pos[1]-Dkkp))/(dist(apos[0],apos[1],n2pos[0],n2pos[1]))))
Zbn1 = Zbn2 =          Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(bpos[0], bpos[1], n1pos[0], n1pos[1]-Dkkp))/(dist(bpos[0],bpos[1],n1pos[0],n1pos[1]))))

Zn1n2 = Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(n1pos[0], n1pos[1], n2pos[0], n2pos[1]-Dkkp))/(dist(n1pos[0],n1pos[1],n2pos[0],n2pos[1]))))

Za = [[Zaa, Zab, Zac],
      [Zab, Zbb, Zbc],
      [Zac, Zbc, Zcc]]

Zb = [[Zan1, Zan2],
      [Zbn1, Zbn2],
      [Zcn1, Zcn2]]

Zc = [[Zan1, Zbn1, Zcn1],
      [Zan2, Zbn2, Zcn2]]

Zd = [[Zn1n1, Zn1n2],
      [Zn1n2, Zn2n2]]

Zp = Za-np.matmul(np.matmul(Zb,np.linalg.inv(Zd)),Zc)

Zs = (Zp[0,0]+Zp[1,1]+Zp[2,2])/3
Zm = (Zp[0,1]+Zp[0,2]+Zp[1,2])/3

In [225]:
Zpsym = [[Zs,Zm,Zm],
         [Zm,Zs,Zm],
         [Zm,Zm,Zs]]

a = cmath.rect(1,120*(np.pi/180))
Ainv=[[1,1,1],
   [1,a,a**2],
   [1,a**2,a]]
A=[[1,1,1],
   [1,a**2,a],
   [1,a,a**2]]

Zshat = (1/3)*np.matmul(np.matmul(Ainv,Zpsym),A)  #ohms per meter
print(f"Z0 = {Zshat[0,0]*1000} ohm/km")
print(f"Z1 = {Zshat[1,1]*1000} ohm/km")
print(f"Z2 = {Zshat[2,2]*1000} ohm/km")

Z0 = (0.45598785298888267+1.2222047446536548j) ohm/km
Z1 = (0.03566769900140839+0.36557678690227485j) ohm/km
Z2 = (0.03566769900140841+0.36557678690227485j) ohm/km


(-7.834466149794468e-17+9.925903512171851e-17j)